# Evolutionary Algorithms - Flight Schedule Optimization

## Task Description
An airline operates a fleet of 8 aircraft with varying capacities and operating costs. Each aircraft has a designated home airport.

### Objective
Develop an evolutionary algorithm that maximizes profit by optimally assigning 30 scheduled flights to the aircraft fleet.

### Problem Constraints
- All flights must be executed - no cancellations allowed, even if unprofitable 
- Capacity limits: Aircraft cannot exceed their seat capacity 
- Scheduled departure: Flights cannot depart early but may depart late 
- Ground time: 40 minutes required between consecutive flights for refueling and service 
- No flight merging: Each flight operates independently

### Profit Calculation
**Revenue:** Ticket price per passenger × number of passengers 

**Costs:**
- Operating costs (per flight hour, varies by aircraft) 
- Delay penalty: €10 per minute for late departures (per flight) 
- Empty positioning flights (if needed to relocate aircraft) 

### Implementation Requirements
1. Design a suitable genome representation for flight-to-aircraft assignments 
2. Implement crossover and mutation operators 
3. Create a fitness evaluation function that calculates total profit 
4. Apply the algorithm to the provided dataset (8 planes, 30 flights, 8 airports)

## 1. Imports

In [57]:
import pandas as pd
import random
from deap import base, creator, tools, algorithms
import matplotlib.pyplot as plt

## 2. Data Exploration & Preprocessing
To make handling of the data easier, we exported all tables of the orginial excel file to .csv files. 

Now we can import the .csv files from the `data/` directory.

In [58]:
# Load dataset
data_flights = pd.read_csv('data/EA_Airline_Data_FLIGHTS.csv')
data_planes = pd.read_csv('data/EA_Airline_Data_PLANES.csv')
data_price = pd.read_csv('data/EA_Airline_Data_PRICE.csv')
data_time = pd.read_csv('data/EA_Airline_Data_TIME.csv')

# Display data
print(data_flights.head())
print("-"*80)
print(data_planes.head())
print("-"*80)
print(data_price.head())
print("-"*80)
print(data_time.head())

   ID Start Ziel  Minuten bis Abflug  Passagiere
0   0   HAM  FMM                2050          53
1   1   FMM  CGN                 954          59
2   2   HAM  BRE                2753          52
3   3   CGN  FDH                1118          53
4   4   CGN  FMM                1358          56
--------------------------------------------------------------------------------
   ID              Typ  Sitzplätze Heimatflughafen  Kosten €/h
0   0   Boeing 737-800         160             STR        6700
1   1       ATR 72-600          75             STR        2500
2   2  Airbus A320-200         174             FMM        6800
3   3     Embraer E190         101             FMM        3000
4   4     Embraer E190         108             BER        3800
--------------------------------------------------------------------------------
  From/To    BER    BRE    CGN    FDH    FMM    HAJ    HAM    STR
0     BER    NaN  122.0  124.0  188.0  198.0  148.0  136.0  106.0
1     BRE  122.0    NaN  108.0  11

### Data Structure Overview

**FLIGHTS Table:** Contains scheduled flights with ID, origin, destination, minutes until departure and passenger count

**PLANES Table:** Contains aircraft information including type, seating capacity, home airport, and operating cost per hour

**PRICE Table:** Matrix of ticket prices between airports (From/To structure)

**TIME Table:** Matrix of flight times in minutes between airports (From/To structure)

### Preprocessing Steps
1. Rename German columns to English for better code readability
2. Convert TIME and PRICE matrices to indexed DataFrames for efficient lookup

In [59]:
# Rename columns in PLANES DataFrame
df_planes = data_planes.rename(columns={
    'Sitzplätze': 'seats',
    'Heimatflughafen': 'home_airport',
    'Kosten €/h': 'cost_per_hour',
    'Typ': 'Type'
})

# Rename columns in FLIGHTS DataFrame
df_flights = data_flights.rename(columns={
    'Minuten bis Abflug': 'minutes_until_departure',
    'Passagiere': 'passengers',
    'Typ': 'Type',
    'Ziel': 'Destination'
})

# Convert TIME and PRICE DataFrames to indexed format
# Set the 'From/To' column as the index for easy lookup
df_time = data_time.set_index('From/To')
df_price = data_price.set_index('From/To')

# Display processed data
print("Planes DataFrame:")
print(df_planes.head())
print("\n" + "-"*80 + "\n")

print("Flights DataFrame:")
print(df_flights.head())
print("\n" + "-"*80 + "\n")

print("Time Matrix (indexed):")
print(df_time.head())
print("\n" + "-"*80 + "\n")

print("Price Matrix (indexed):")
print(df_price.head())

Planes DataFrame:
   ID             Type  seats home_airport  cost_per_hour
0   0   Boeing 737-800    160          STR           6700
1   1       ATR 72-600     75          STR           2500
2   2  Airbus A320-200    174          FMM           6800
3   3     Embraer E190    101          FMM           3000
4   4     Embraer E190    108          BER           3800

--------------------------------------------------------------------------------

Flights DataFrame:
   ID Start Destination  minutes_until_departure  passengers
0   0   HAM         FMM                     2050          53
1   1   FMM         CGN                      954          59
2   2   HAM         BRE                     2753          52
3   3   CGN         FDH                     1118          53
4   4   CGN         FMM                     1358          56

--------------------------------------------------------------------------------

Time Matrix (indexed):
          BER   BRE   CGN   FDH   FMM   HAJ   HAM   STR
From

## 3. Genome Representation
Each flight and each plane has an assigned ID. To assign planes to flights, we represent the assignments with an array of length `num_flights`. The entry at index `i` is the ID of the plane assigned to flight `i`.



In [60]:
# Constants
NUM_FLIGHTS = len(df_flights)
NUM_PLANES = len(df_planes)

print(f"Number of flights: {NUM_FLIGHTS}")
print(f"Number of planes: {NUM_PLANES}")
print()

# Example genome representation
# Each index represents a flight ID, and the value is the assigned plane ID
example_genome = [2, 5, 1, 4, 7, 0, 3, 6, 2, 1, 5, 4, 0, 7, 3, 6, 2, 5, 1, 4, 7, 0, 3, 6, 2, 1, 5, 4, 0, 7]

print(f"Example genome: {example_genome}")
print(f"Genome length: {len(example_genome)}")
print()
print("Interpretation:")
print(f"  Flight 0 is assigned to Plane {example_genome[0]}")
print(f"  Flight 1 is assigned to Plane {example_genome[1]}")
print(f"  Flight 2 is assigned to Plane {example_genome[2]}")
print("  ...")

Number of flights: 30
Number of planes: 8

Example genome: [2, 5, 1, 4, 7, 0, 3, 6, 2, 1, 5, 4, 0, 7, 3, 6, 2, 5, 1, 4, 7, 0, 3, 6, 2, 1, 5, 4, 0, 7]
Genome length: 30

Interpretation:
  Flight 0 is assigned to Plane 2
  Flight 1 is assigned to Plane 5
  Flight 2 is assigned to Plane 1
  ...


## 4. Helper Functions
To simplify subsequent processing steps, several helper functions are defined:

### 4.1 Flight/Price Lookup

In [61]:
def get_flight_time(origin, destination):
    """
    Get flight time in minutes between two airports.
    
    Args:
        origin (str): Origin airport code (e.g., 'HAM')
        destination (str): Destination airport code (e.g., 'FMM')
    
    Returns:
        float: Flight time in minutes, or 0 if same airport
    """
    if origin == destination:
        return 0
    return df_time.loc[origin, destination]


def get_ticket_price(origin, destination):
    """
    Get ticket price between two airports.
    
    Args:
        origin (str): Origin airport code (e.g., 'HAM')
        destination (str): Destination airport code (e.g., 'FMM')
    
    Returns:
        float: Ticket price in euros, or 0 if same airport
    """
    if origin == destination:
        return 0
    return df_price.loc[origin, destination]

In [62]:
# Test flight/price lookup functions
print("="*80)
print("Testing flight/price lookup functions:")
print("="*80)
print(f"Flight time HAM -> FMM: {get_flight_time('HAM', 'FMM')} minutes")
print(f"Ticket price HAM -> FMM: €{get_ticket_price('HAM', 'FMM')}")
print(f"Flight time BER -> CGN: {get_flight_time('BER', 'CGN')} minutes")
print(f"Ticket price BER -> CGN: €{get_ticket_price('BER', 'CGN')}")

Testing flight/price lookup functions:
Flight time HAM -> FMM: 66.0 minutes
Ticket price HAM -> FMM: €150.0
Flight time BER -> CGN: 55.0 minutes
Ticket price BER -> CGN: €124.0


### 4.2 Genome Utility

In [63]:
def create_random_genome():
    """
    Create a random genome representing flight-to-plane assignments.
    
    Returns:
        list: A list of 30 random integers between 0 and 7 (inclusive),
              where index i represents flight i and value is the assigned plane ID
    """
    return [random.randint(0, NUM_PLANES - 1) for _ in range(NUM_FLIGHTS)]

def is_valid_genome(genome):
    """
    Check if a genome is valid.
    
    Args:
        genome (list): The genome to validate
    
    Returns:
        bool: True if genome is a list of length NUM_FLIGHTS with values 0 to NUM_PLANES-1,
              False otherwise
    """
    if not isinstance(genome, list):
        return False
    
    if len(genome) != NUM_FLIGHTS:
        return False
    
    for gene in genome:
        if not isinstance(gene, int) or gene < 0 or gene >= NUM_PLANES:
            return False
    
    return True


def get_plane_flights(genome, plane_id):
    """
    Get all flight indices assigned to a specific plane.
    
    Args:
        genome (list): The genome representing flight-to-plane assignments
        plane_id (int): The plane ID (0 to NUM_PLANES-1)
    
    Returns:
        list: List of flight indices (integers) assigned to the given plane
    """
    return [flight_idx for flight_idx, assigned_plane in enumerate(genome) if assigned_plane == plane_id]

In [64]:
# Test genome functions
print("\n" + "="*80)
print("Testing genome functions:")
print("="*80)

# Test create_random_genome
random_genome = create_random_genome()
print(f"Random genome: {random_genome}")
print(f"Random genome length: {len(random_genome)}")

# Test is_valid_genome
print(f"\nIs random genome valid? {is_valid_genome(random_genome)}")
print(f"Is [1,2,3] valid? {is_valid_genome([1, 2, 3])}")  # Too short
print(f"Is genome with value 10 valid? {is_valid_genome([0]*29 + [10])}")  # Invalid plane ID

# Test get_plane_flights
test_genome = [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]
print(f"\nTest genome: {test_genome}")
print(f"Flights assigned to Plane 0: {get_plane_flights(test_genome, 0)}")
print(f"Flights assigned to Plane 1: {get_plane_flights(test_genome, 1)}")
print(f"Flights assigned to Plane 2: {get_plane_flights(test_genome, 2)}")
print(f"Flights assigned to Plane 7: {get_plane_flights(test_genome, 7)}")


Testing genome functions:
Random genome: [0, 0, 4, 5, 5, 2, 6, 5, 6, 3, 0, 2, 6, 4, 4, 5, 1, 0, 4, 1, 7, 7, 1, 3, 7, 6, 7, 7, 0, 4]
Random genome length: 30

Is random genome valid? True
Is [1,2,3] valid? False
Is genome with value 10 valid? False

Test genome: [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]
Flights assigned to Plane 0: [0, 3, 6, 9, 12, 15, 18, 21, 24, 27]
Flights assigned to Plane 1: [1, 4, 7, 10, 13, 16, 19, 22, 25, 28]
Flights assigned to Plane 2: [2, 5, 8, 11, 14, 17, 20, 23, 26, 29]
Flights assigned to Plane 7: []


### 4.3 Fitness Calculation

In [65]:
def calculate_flight_revenue(flight_id):
    """
    Calculate revenue for a scheduled flight.
    
    Args:
        flight_id (int): The ID of the flight
    
    Returns:
        float: Total revenue (passengers × ticket_price) in euros
    """
    flight = df_flights.loc[flight_id]
    passengers = flight['passengers']
    origin = flight['Start']
    destination = flight['Destination']
    ticket_price = get_ticket_price(origin, destination)
    
    return passengers * ticket_price


def calculate_flight_cost(plane_id, origin, destination):
    """
    Calculate operating cost for a flight.
    
    Args:
        plane_id (int): The ID of the plane
        origin (str): Origin airport code
        destination (str): Destination airport code
    
    Returns:
        float: Operating cost in euros (cost_per_hour / 60 × flight_time_minutes)
    """
    flight_time_minutes = get_flight_time(origin, destination)
    cost_per_hour = df_planes.loc[plane_id, 'cost_per_hour']
    cost_per_minute = cost_per_hour / 60.0
    
    return cost_per_minute * flight_time_minutes


def calculate_empty_flight_cost(plane_id, origin, destination):
    """
    Calculate operating cost for an empty repositioning flight.
    This is the same as calculate_flight_cost since operating costs don't depend on passengers.
    
    Args:
        plane_id (int): The ID of the plane
        origin (str): Origin airport code
        destination (str): Destination airport code
    
    Returns:
        float: Operating cost in euros for the empty flight
    """
    return calculate_flight_cost(plane_id, origin, destination)


def get_sorted_flights_for_plane(genome, plane_id):
    """
    Get all flights assigned to a plane, sorted by scheduled departure time.
    
    Args:
        genome (list): The genome representing flight-to-plane assignments
        plane_id (int): The ID of the plane
    
    Returns:
        list: List of tuples (flight_id, scheduled_time, origin, destination, passengers)
              sorted by scheduled_time (earliest first)
    """
    flight_indices = get_plane_flights(genome, plane_id)
    
    flights_info = []
    for flight_id in flight_indices:
        flight = df_flights.loc[flight_id]
        scheduled_time = flight['minutes_until_departure']
        origin = flight['Start']
        destination = flight['Destination']
        passengers = flight['passengers']
        
        flights_info.append((flight_id, scheduled_time, origin, destination, passengers))
    
    # Sort by scheduled time (index 1 in tuple)
    flights_info.sort(key=lambda x: x[1])
    
    return flights_info

In [66]:
# Test fitness calculation functions
print("\n" + "="*80)
print("Testing fitness calculation functions:")
print("="*80)

# Test calculate_flight_revenue
flight_0 = df_flights.loc[0]
print(f"\nFlight 0: {flight_0['Start']} -> {flight_0['Destination']}, {flight_0['passengers']} passengers")
revenue_0 = calculate_flight_revenue(0)
print(f"Revenue for Flight 0: €{revenue_0:.2f}")

# Test calculate_flight_cost
plane_0 = df_planes.loc[0]
print(f"\nPlane 0: {plane_0['Type']}, Cost: €{plane_0['cost_per_hour']}/hour")
cost_0 = calculate_flight_cost(0, 'HAM', 'FMM')
print(f"Operating cost for Plane 0 (HAM -> FMM): €{cost_0:.2f}")

# Test calculate_empty_flight_cost
empty_cost = calculate_empty_flight_cost(0, 'STR', 'HAM')
print(f"\nEmpty repositioning flight cost for Plane 0 (STR -> HAM): €{empty_cost:.2f}")

# Test get_sorted_flights_for_plane
test_genome = create_random_genome()
sorted_flights = get_sorted_flights_for_plane(test_genome, 0)
print(f"\nPlane 0 has {len(sorted_flights)} flights assigned:")
if sorted_flights:
    print("First 3 flights (sorted by scheduled time):")
    for flight_info in sorted_flights[:3]:
        flight_id, scheduled_time, origin, dest, passengers = flight_info
        print(f"  Flight {flight_id}: {origin}->{dest}, {passengers} pax, departs in {scheduled_time} min")


Testing fitness calculation functions:

Flight 0: HAM -> FMM, 53 passengers
Revenue for Flight 0: €7950.00

Plane 0: Boeing 737-800, Cost: €6700/hour
Operating cost for Plane 0 (HAM -> FMM): €7370.00

Empty repositioning flight cost for Plane 0 (STR -> HAM): €8933.33

Plane 0 has 5 flights assigned:
First 3 flights (sorted by scheduled time):
  Flight 7: BER->CGN, 45 pax, departs in 221 min
  Flight 3: CGN->FDH, 53 pax, departs in 1118 min
  Flight 14: BRE->CGN, 49 pax, departs in 1500 min


## 5. Fitness Function

The fitness function evaluates how profitable a given genome (flight-to-plane assignment) is by calculating the total profit.

### Calculation Process

For each plane in the fleet, the function:
1. **Retrieves assigned flights** sorted by scheduled departure time
2. **Tracks plane state** (current location and time) throughout the schedule
3. **Calculates repositioning costs** when the plane needs to fly empty to reach the next flight's origin
4. **Enforces constraints:**
   - 40-minute turnaround time between consecutive flights
   - Flights cannot depart early (only on-time or delayed)
   - Capacity violations penalized heavily
5. **Computes profit components:**
   - **Revenue:** Ticket price × number of passengers
   - **Operating costs:** Based on plane type and flight duration
   - **Delay penalties:** €10 per minute for late departures
   - **Empty flight costs:** Repositioning flights with no revenue

### Fitness Value
The function returns the total profit (which can be negative if costs exceed revenue). Higher fitness values indicate better flight assignments.

In [67]:
GROUND_TIME = 40  # Minutes required for refueling and service
DELAY_PENALTY = 10  # Euros per minute of delay
CAPACITY_PENALTY = 100000  # Large penalty for exceeding capacity


def calculate_fitness(genome):
    """
    Calculate total profit for a given genome (flight-to-plane assignment).
    
    Args:
        genome (list): List of 30 integers representing plane assignments for each flight
    
    Returns:
        tuple: (total_profit,) - DEAP requires fitness to be a tuple
    """
    total_profit = 0.0
    
    # Initialize state for each plane: current location and time
    plane_states = {}
    for plane_id in range(NUM_PLANES):
        home_airport = df_planes.loc[plane_id, 'home_airport']
        plane_states[plane_id] = {
            'current_location': home_airport,
            'current_time': 0
        }
    
    # Process each plane's schedule
    for plane_id in range(NUM_PLANES):
        # Get all flights assigned to this plane, sorted by scheduled time
        sorted_flights = get_sorted_flights_for_plane(genome, plane_id)
        
        # Track if this is the first flight for this plane
        is_first_flight = True
        
        for flight_id, scheduled_time, origin, destination, passengers in sorted_flights:
            plane_state = plane_states[plane_id]
            current_location = plane_state['current_location']
            current_time = plane_state['current_time']
            
            # Calculate repositioning time if plane is not at flight origin
            repositioning_time = 0
            if current_location != origin:
                repositioning_time = get_flight_time(current_location, origin)
                # Add cost of empty repositioning flight
                empty_flight_cost = calculate_empty_flight_cost(plane_id, current_location, origin)
                total_profit -= empty_flight_cost
            
            # Calculate earliest possible departure time
            # First flight doesn't need ground time, subsequent flights do
            turnaround_time = 0 if is_first_flight else GROUND_TIME
            earliest_possible_departure = current_time + repositioning_time + turnaround_time
            
            # Calculate delay (can't depart early, only late)
            delay = max(0, earliest_possible_departure - scheduled_time)
            
            # Add delay penalty
            total_profit -= delay * DELAY_PENALTY
            
            # Check capacity constraint
            plane_seats = df_planes.loc[plane_id, 'seats']
            if passengers > plane_seats:
                # Large penalty for exceeding capacity
                total_profit -= CAPACITY_PENALTY
            
            # Add flight revenue
            revenue = calculate_flight_revenue(flight_id)
            total_profit += revenue
            
            # Subtract flight operating cost
            flight_cost = calculate_flight_cost(plane_id, origin, destination)
            total_profit -= flight_cost
            
            # Update plane state
            actual_departure = max(earliest_possible_departure, scheduled_time)
            flight_time = get_flight_time(origin, destination)
            plane_state['current_location'] = destination
            plane_state['current_time'] = actual_departure + flight_time
            
            # Mark that first flight has been processed
            is_first_flight = False
    
    return (total_profit,)  # DEAP requires tuple

In [68]:
# Test fitness function
print("="*80)
print("Testing Fitness Function")
print("="*80)

# Test 1: Random genome
print("\nTest 1: Random genome")
test_genome_1 = create_random_genome()
print(f"Genome: {test_genome_1}")
fitness_1 = calculate_fitness(test_genome_1)
print(f"Total profit: €{fitness_1[0]:,.2f}")

# Test 2: All flights assigned to plane 0 (will likely have capacity violations)
print("\nTest 2: All flights to Plane 0 (capacity test)")
test_genome_2 = [0] * NUM_FLIGHTS
fitness_2 = calculate_fitness(test_genome_2)
print(f"Total profit: €{fitness_2[0]:,.2f}")
print("(Expected: Large negative value due to capacity violations)")

# Test 3: Evenly distributed flights
print("\nTest 3: Evenly distributed flights")
test_genome_3 = [i % NUM_PLANES for i in range(NUM_FLIGHTS)]
print(f"Genome: {test_genome_3}")
fitness_3 = calculate_fitness(test_genome_3)
print(f"Total profit: €{fitness_3[0]:,.2f}")

# Test 4: Compare multiple random genomes
print("\nTest 4: Comparing 5 random genomes")
print("-" * 80)
fitness_values = []
for i in range(5):
    genome = create_random_genome()
    fitness = calculate_fitness(genome)
    fitness_values.append(fitness[0])
    print(f"Genome {i+1} profit: €{fitness[0]:,.2f}")

print(f"\nBest profit: €{max(fitness_values):,.2f}")
print(f"Worst profit: €{min(fitness_values):,.2f}")
print(f"Average profit: €{sum(fitness_values)/len(fitness_values):,.2f}")

Testing Fitness Function

Test 1: Random genome
Genome: [0, 4, 5, 2, 1, 7, 0, 7, 4, 7, 3, 5, 4, 7, 6, 6, 1, 0, 4, 1, 4, 2, 1, 4, 1, 2, 6, 7, 2, 7]
Total profit: €-80,134.67

Test 2: All flights to Plane 0 (capacity test)
Total profit: €-429,391.33
(Expected: Large negative value due to capacity violations)

Test 3: Evenly distributed flights
Genome: [0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5]
Total profit: €-90,589.67

Test 4: Comparing 5 random genomes
--------------------------------------------------------------------------------
Genome 1 profit: €-108,304.67
Genome 2 profit: €-62,248.00
Genome 3 profit: €-81,371.33
Genome 4 profit: €-83,641.33
Genome 5 profit: €-91,294.67

Best profit: €-62,248.00
Worst profit: €-108,304.67
Average profit: €-85,372.00


## 6. DEAP Setup & Evolutionary Algorithm

We use DEAP (Distributed Evolutionary Algorithms in Python) framework to implement the evolutionary algorithm.

### 6.1 DEAP Configuration

Set up DEAP's creator and toolbox with our problem-specific functions.

In [79]:
# Create fitness and individual classes using DEAP's creator
# FitnessMax: we want to MAXIMIZE profit
if not hasattr(creator, "FitnessMax"):
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))

if not hasattr(creator, "Individual"):
    creator.create("Individual", list, fitness=creator.FitnessMax)

# Create toolbox and register functions
toolbox = base.Toolbox()

# Register genome creation: random plane ID (0-7) for each flight
toolbox.register("plane_id", random.randint, 0, NUM_PLANES - 1)

# Register individual creation: list of 30 random plane assignments
toolbox.register("individual", tools.initRepeat, creator.Individual, 
                 toolbox.plane_id, NUM_FLIGHTS)

# Register population creation
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Register genetic operators
toolbox.register("evaluate", calculate_fitness)
toolbox.register("mate", tools.cxTwoPoint)  # Two-point crossover
toolbox.register("mutate", tools.mutUniformInt, low=0, up=NUM_PLANES-1, indpb=0.1)  # Uniform mutation
toolbox.register("select", tools.selTournament, tournsize=3)  # Tournament selection

print("DEAP Toolbox configured successfully!")
print(f"- Fitness: Maximize profit")
print(f"- Individual: List of {NUM_FLIGHTS} plane assignments (0-{NUM_PLANES-1})")
print(f"- Crossover: Two-point")
print(f"- Mutation: Uniform integer (10% per gene)")
print(f"- Selection: Tournament (size=3)")

DEAP Toolbox configured successfully!
- Fitness: Maximize profit
- Individual: List of 30 plane assignments (0-7)
- Crossover: Two-point
- Mutation: Uniform integer (10% per gene)
- Selection: Tournament (size=3)


### 6.2 Testing DEAP Operators

Test the registered DEAP operators to verify they work correctly.

In [80]:
# Test individual creation
print("Creating individuals using DEAP:")
ind1 = toolbox.individual()
ind2 = toolbox.individual()
print(f"Individual 1: {ind1}")
print(f"Individual 2: {ind2}")
print(f"Individual type: {type(ind1)}")
print(f"Fitness type: {type(ind1.fitness)}")

Creating individuals using DEAP:
Individual 1: [4, 0, 3, 2, 6, 4, 3, 4, 1, 0, 1, 4, 7, 7, 1, 2, 2, 1, 7, 7, 2, 4, 2, 5, 6, 6, 1, 7, 6, 3]
Individual 2: [3, 5, 3, 7, 0, 5, 4, 6, 7, 0, 1, 2, 2, 4, 0, 1, 1, 1, 2, 4, 4, 3, 2, 4, 7, 4, 2, 4, 5, 3]
Individual type: <class 'deap.creator.Individual'>
Fitness type: <class 'deap.creator.FitnessMax'>


In [85]:
# Test population creation
test_pop = toolbox.population(n=5)
print("\nSample population (5 individuals):")
for i, individual in enumerate(test_pop):
    print(f"Individual {i+1}: {list(individual)}")


Sample population (5 individuals):
Individual 1: [6, 3, 0, 1, 4, 5, 5, 1, 0, 7, 4, 6, 6, 4, 4, 3, 5, 4, 4, 6, 2, 7, 6, 4, 4, 7, 5, 6, 3, 7]
Individual 2: [3, 7, 7, 3, 6, 7, 0, 0, 6, 6, 6, 7, 5, 5, 2, 0, 1, 6, 5, 4, 0, 4, 7, 0, 5, 4, 6, 5, 0, 6]
Individual 3: [5, 5, 2, 1, 1, 2, 0, 0, 6, 2, 4, 5, 0, 2, 6, 5, 1, 3, 2, 0, 2, 7, 7, 1, 4, 4, 5, 6, 4, 3]
Individual 4: [3, 5, 7, 0, 5, 2, 0, 6, 4, 6, 6, 1, 5, 4, 1, 0, 0, 7, 5, 7, 6, 6, 1, 1, 5, 2, 2, 1, 1, 1]
Individual 5: [3, 3, 0, 6, 3, 7, 5, 6, 2, 2, 3, 2, 2, 1, 2, 5, 3, 7, 0, 0, 5, 6, 2, 4, 6, 3, 3, 3, 6, 4]


### 6.3 Test Selection

Test DEAP's tournament selection operator.

In [92]:
# Test selection
test_pop = toolbox.population(n=5)

# Evaluate population
for ind in test_pop:
    ind.fitness.values = toolbox.evaluate(ind)

print("Population fitnesses:")
for i, ind in enumerate(test_pop):
    print(f"Individual {i+1}: €{ind.fitness.values[0]:,.2f}")

# Select one individual using tournament selection
selected = toolbox.select(test_pop, 1)
print(f"\nSelected individual fitness: €{selected[0].fitness.values[0]:,.2f}")

# Select multiple individuals
selected_multiple = toolbox.select(test_pop, 3)
print(f"\nSelected 3 individuals:")
for i, ind in enumerate(selected_multiple):
    print(f"  Individual {i+1}: €{ind.fitness.values[0]:,.2f}")

Population fitnesses:
Individual 1: €-84,173.00
Individual 2: €-85,276.33
Individual 3: €-59,204.67
Individual 4: €-64,428.00
Individual 5: €-113,114.67

Selected individual fitness: €-64,428.00

Selected 3 individuals:
  Individual 1: €-59,204.67
  Individual 2: €-64,428.00
  Individual 3: €-59,204.67


### 6.4 Test Crossover

Test DEAP's two-point crossover operator.

In [93]:
# Test crossover
parent1 = toolbox.individual()
parent2 = toolbox.individual()

# Create simple test parents for visibility
for i in range(NUM_FLIGHTS):
    parent1[i] = 0
    parent2[i] = 7

print(f"Parent 1: {list(parent1)}")
print(f"Parent 2: {list(parent2)}")

# Perform crossover (modifies parents in-place)
offspring1, offspring2 = toolbox.clone(parent1), toolbox.clone(parent2)
toolbox.mate(offspring1, offspring2)

print(f"\nOffspring 1: {list(offspring1)}")
print(f"Offspring 2: {list(offspring2)}")
print("(Notice the middle segment is swapped)")

Parent 1: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Parent 2: [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]

Offspring 1: [0, 0, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0, 0, 0, 0, 0, 0, 0]
Offspring 2: [7, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 7, 7, 7, 7, 7, 7]
(Notice the middle segment is swapped)


### 6.5 Test Mutation

Test DEAP's uniform integer mutation operator.

In [94]:
# Test mutation
original = toolbox.individual()
for i in range(NUM_FLIGHTS):
    original[i] = 0

print(f"Original genome: {list(original)}")

# Clone and mutate
mutated = toolbox.clone(original)
toolbox.mutate(mutated)

print(f"Mutated genome: {list(mutated)}")

# Count differences
differences = sum(1 for i in range(len(original)) if original[i] != mutated[i])
print(f"\nNumber of genes mutated: {differences}/{NUM_FLIGHTS} ({differences/NUM_FLIGHTS*100:.1f}%)")

Original genome: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Mutated genome: [0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Number of genes mutated: 2/30 (6.7%)


## 7. Main Evolutionary Algorithm Loop

## 8. Base Line Results

## 9. Scenario Experiments

## 10. Evaluation